The recent paper from Besley, Fontana and Limodio (2020, AEJ Insights) uses a database that is avaiable digitally but not in a format able to be read in actively as a .do file. I scrape Hylton and Deng's Antitrust Database-- which is used in the paper, with this code. Note. Various measures of anti-trust that go into the rankings/scores are not scraped. This must be worked on. 

In [ ]:
!pip3 install country_converter

     |████████████████████████████████| 51kB 2.8MB/s 
  Created wheel for country-converter: filename=country_converter-0.7.1-cp36-none-any.whl size=50807 sha256=c5d17b8f46e8f4e87768dd867c318c72ccc9cc1ec674d6939f0c9e2ce87db291
  Stored in directory: /root/.cache/pip/wheels/75/be/cb/3c8d07bca8f2c0670e996a174fed372585b32d3aa03f40996a
Successfully built country-converter


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urlparse, urljoin
import urllib
import  nltk
import webbrowser
import geograpy
import pycountry
import pandas as pd
from urllib.request import urlopen
from geograpy import places
from geograpy import extraction
import scrapy
import re
from dateutil.parser import parse
import calendar
from string import digits
from collections import OrderedDict
import datetime
import country_converter as coco


In [ ]:
res = requests.get('http://antitrustworldwiki.com/antitrustwiki/index.php/Main_Page')
soup = bs(res.text, "html.parser")
wiki_dict = {}
linklist = []
for link in soup.find_all("a"):
    url = link.get("href", "")
    if "antitrustwiki/index.php/" in url:
        wiki_dict[link.text.strip()] = url
        linklist.append(wiki_dict)
countrylist = pd.DataFrame.from_dict(linklist)
#pycountries do not recognize these irregular naming conventions
columns= list(country.name for country in pycountry.countries)
exceptions = ["Bolivia", "Venezuela", "Cote d'Ivoire", 'Vietnam', 'Iran',
              'South Korea','Sudan','Macau','Republic of Ireland', 
              'Slovak Republic', 'Bosnia-Herzegovina', 'Lao PDR', 'Tanzania', 'Taiwan',
              "Macedonia", "Russia", "Moldova", "Faroe Island" "Czech Republic"]
def Union(lst1, lst2): 
    final_list = lst1 + lst2 
    return final_list 
united_columns = Union(columns, exceptions)


In [ ]:
countrylist = countrylist[countrylist.columns.intersection(united_columns)]
countrylist= countrylist.drop_duplicates()
country_url =  countrylist.values.tolist()[0] 
string = "http://antitrustworldwiki.com"
list_url = [string+str(x) for x in country_url]

In [ ]:
wiki_dict2 = {}
linklist2 = []
for l in list_url:
  r = requests.get(l)
  soup = bs(r.text, "html.parser")
  for link in soup.find_all("a"):
    url2 = link.get("href", "")
    if "antitrustwiki/index.php/" in url2:
        wiki_dict2[link.text.strip()] = url2
        linklist2.append(wiki_dict2)
countrylist2 = pd.DataFrame.from_dict(linklist2)

In [ ]:
countrylist2= countrylist2.drop_duplicates()
country_url2 =  countrylist2.values.tolist()[0] 
list_url2 = [string+str(x) for x in country_url2]
linklist3 = []
for x in list_url2:
  for y in list_url:
    if y in x:
      linklist3.append(x)
linklist3 = [x for x in linklist3 if "/EU," not in x]

In [ ]:
year=[]
appended=[]
dicts={}
for z in linklist3:
  req = requests.get(z)
  soup3 = bs(req.text, 'html.parser')
  for nat in soup3.find_all('h1'):
    for line in soup3.find_all('b'):
      if not "Score" in line.text:continue  
      appended.append([nat.get_text(strip=True), line.get_text(strip=True)])


In [ ]:
appended

[['Austria (January 1, 2006)', 'Score = 19'],
 ['Austria (January 1, 2000)', 'Score = 19'],
 ['Belgium, 2006', 'Score = 20'],
 ['Belgium 1999', 'Score = 18'],
 ['Bulgaria 2003', 'Score = 20'],
 ['Bulgaria 1998', 'Score = 17'],
 ['Cyprus (2008)', 'Score = 17'],
 ['Cyprus (1999)', 'Score = 23'],
 ['Denmark July 2000', 'Score = 22'],
 ['Denmark June 2002', 'Score = 22'],
 ['Denmark August 2005', 'Score = 22'],
 ['Denmark August 21, 2007', 'Score = 22'],
 ['Estonia (July 1, 2006)', 'Score = 21'],
 ['Estonia (June 28, 2004)', 'Score = 21'],
 ['Estonia (March 8, 2003)', 'Score = 20'],
 ['Estonia (October 1, 2001)', 'Score = 21'],
 ['Estonia (October 1, 1998)', 'Score = 20'],
 ['Finland 1998', 'Score = 21'],
 ['Finland 2004', 'Score = 19'],
 ['France, 2005', 'Score = 26'],
 ['France, 1999', 'Score = 18'],
 ['France, 1986', 'Score = 19'],
 ['France, 1977', 'Score = 16'],
 ['France, 1967', 'Score = 13'],
 ['France, 1953', 'Score = 12'],
 ['Germany (December 19, 2000)', 'Score = 19'],
 ['Germany

In [ ]:
apd = pd.DataFrame.from_dict(appended)
apd[1] = apd1[1].map(lambda x: x.replace('=','').replace('Score','').strip(' '))
months = []
for i in range(1,12):
  months.append(calendar.month_name[i])
for x in months:
  apd[0] = apd[0].str.replace(x, "")
apd[0] = apd[0].str.replace(",", "")
apd[0] = apd[0].str.replace(")", "")
apd[0] = apd[0].str.replace("(", "")
apd

,0,1
0,Austria 1 2006,19
1,Austria 1 2000,19
2,Belgium 2006,20
3,Belgium 1999,18
4,Bulgaria 2003,20
...,...,...
180,Guyana 2004,15
181,Peru 3 1998,12
182,Uruguay 2007,22
183,Uruguay 2001,15


In [ ]:
apd[3] = apd[0].str.extract("(\d\d\d\d)")
apd[0] = apd[0].str.replace('\d+', '')


There is certainly a better way to do this... Code to be optimized

In [ ]:
apd= apd.rename(columns={1: "Score", 0: "Country", 3: "Year" })
apd["Country"] = apd["Country"].str.replace("December", "")
apd["Country"] = apd["Country"].str.replace("February", "")
apd["Country"] = apd["Country"].str.replace("January", "")
apd["Country"] = apd["Country"].str.replace("March", "")
apd["Country"] = apd["Country"].str.replace("April", "")
apd["Country"] = apd["Country"].str.replace("May", "")
apd["Country"] = apd["Country"].str.replace("June", "")
apd["Country"] = apd["Country"].str.replace("July", "")
apd["Country"] = apd["Country"].str.replace("August", "")
apd["Country"] = apd["Country"].str.replace("September", "")
apd["Country"] = apd["Country"].str.replace("October", "")
apd["Country"] = apd["Country"].str.replace("November", "")
apd["Country"] = apd["Country"].str.replace("Hungary Febuary", "Hungary")
apd["Country"] = apd["Country"].str.replace("Jan", "")
apd["Country"] = apd["Country"].str.replace("Serbia-Montenegro", "Serbia")
apd["Country"] = apd["Country"].str.replace(".", "")
apd["Country"] = apd["Country"].str.replace("Bosnia-Herzegovina", "Bosnia and Herzegovina")
apd["Country"] = apd["Country"].str.replace("Lao PDF", "Laos")
apd["Country"] = apd["Country"].str.replace("South Korea", "Korea, Republic of")
apd["Country"] = apd["Country"].str.replace("Aug", "")
apd


,Country,Score,Year
0,Austria,19,2006
1,Austria,19,2000
2,Belgium,20,2006
3,Belgium,18,1999
4,Bulgaria,20,2003
...,...,...,...
180,Guyana,15,2004
181,Peru,12,1998
182,Uruguay,22,2007
183,Uruguay,15,2001


In [ ]:
def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
    except:
        return print('An exception flew by!')
        result = "LAO"
        raise
    else:
        return result[0].alpha_3
apd["Country"] = apd["Country"].apply(lambda country: do_fuzzy_search(country))


An exception flew by!


In [ ]:
apd.to_excel('antitrust.xlsx', index=False, encoding='utf-8')
apd

from google.colab import files
files.download('antitrust.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>